観測行列$\bf{Y}\in\bf{R_+}^{m\times n}$, 辞書行列$\bf{H}\in\bf{R_+}^{m\times k}$, 係数行列$\bf{H}\in\bf{R_+}^{k\times n}$とする。ただし$k\leq m$

以下のクラスは観測行列$\bf{Y}$と辞書の要素数$k$を取り、$$\bf{Y}=\bf{H}\bf{U}$$となる$\bf{U}$と$\bf{H}$を見つけるNMFを計算するクラスである。

In [3]:
import numpy as np
import math as math

class NMF(object):
    ''' caliculates NMF'''
    ''' requires 2-dimentional numpy array'''
    def __init__(self, Y, k):
        assert len(np.shape(Y)) == 2, 'Matrix size error (shold be 2-d array)'
        self.m = np.shape(Y)[0] 
        self.n = np.shape(Y)[1]
        self.Y = Y
        self.k = k
        self.__initialize()
    '''update dictionary and coefficient'''
    def update(self):
        # aliases
        H = self.H
        U = self.U
        Y = self.Y
        self.H = H*Y.dot(U.T)/H.dot(U.dot(U.T))
        H = self.H
        self.U = U*H.T.dot(Y)/H.T.dot(H).dot(U)
    '''initialize dictionary and coefficient'''
    def __initialize(self): 
        self.H = self.Y[:,:self.k].copy()
        self.U = np.random.ranf(self.k*self.n).reshape(self.k,self.n)            
    '''returns dictionary'''
    def get_dic(self):
        return self.H
    '''returns coefficient'''
    def get_coef(self):
        return self.U
    '''evaluate error by '''
    def error(self):
        diff=self.Y-self.H.dot(self.U)
        return math.sqrt(sum(sum(diff*diff)))
    
 

下記の関数は、$\bf{U}$が列に丁度$n\_sparse$個の要素を含むような行列とし、$\bf{H}$を乱数としたときの$\bf{Y}=\bf{H}\bf{U}$を計算する。$\bf{Y}$はスパースな成分に分解できる行列となる。

In [8]:
''' utirity functions for NMF evaluation '''
def NMF_eval( m=20, n=1500, k=15, n_sparse = 3):

    ''' generate KxN matrix with (n_sparse)-sparse '''
    def create_sparse(k, n, n_sparse):
        elem = np.random.ranf(n_sparse*n).reshape(n_sparse,n)
        zeros = (np.zeros((k-n_sparse)*n).reshape((k-n_sparse),n))
        X = np.concatenate((elem, zeros), axis=0)
        X = [np.random.permutation(x) for x in np.transpose(X)]
        return np.transpose(X)

    
    ''' generate matrix contains sparse components '''
    ''' X = UY, U=random, Y=sparse matrix'''
    def create_data(m,n,k,n_sparse):
        U = create_sparse(k,n,n_sparse)
        H = np.random.ranf(m*k).reshape(m,k) # R(KxM)
        Y = H.dot(U)
        return Y,H,U
    
    def evaluate(Y, H, U, trueDic):
        def count_atoms(_est_A, _true_A, axis=0):
            from sklearn.preprocessing import normalize
            """ Count recovered atoms
            Parameters
            ----------
            _est_A : array, shape(n_features, n_samples)
                estimated matrix
            _true_A : array, shape(n_features, n_samples)
                true matrix
            axis : 0 or 1, optional (1 by default)
                axis used to normalize the data along.
                If 1,            independently normalize each sample,
                otherwise (if 0) normalize each feature.
            """

            if not (_est_A.shape == _true_A.shape):
                raise ValueError("The shape of dictionaries should be same;"
                                 "got %r and %r " % (_est_A.shape, _true_A.shape))
            num_recovered = 0
            num_atoms = len(_true_A[0])

            est_A = normalize(_est_A, axis=axis)
            true_A = normalize(_true_A, axis=axis)

            for e in est_A.T:

                distances = [np.mean((e - t) ** 2) for t in true_A.T]

                min_idx = np.argmin(distances)
                min_t = true_A[:, min_idx]

                # Assume normalization for atoms (Aatom' * Eatom = 1)
                dis_t = 1 - abs(np.dot(min_t.T, e))

                if (dis_t < 0.01):
                    num_recovered += 1

            recovered_rate = 100 * (num_recovered / num_atoms)
            return recovered_rate
    
    # initialize
    error = [];
    atom = [];
    Y,trueH,trueU = create_data(m,n,k, n_sparse)
    res = NMF(Y,k)
    print(np.shape(Y), np.shape(trueH), np.shape(trueU))
    print('test')
    # update dictionary and coefficients
    for i in range(0,2000):
        res.update()
        H = res.get_dic()
        U = res.get_coef()
        err, atm = evaluate(Y, H, U, trueH)
        error.append(err)
        atom.append(atm)
        print(err,atm)
        
    #return error, atom

In [9]:
NMF_eval()

(20, 1500) (20, 15) (15, 1500)
test
1938.74497985 0.0
1880.9124427 0.0
1824.17508439 0.0
1765.2140213 0.0
1702.3710193 0.0
1634.91140231 0.0
1563.01765496 0.0
1487.66728393 0.0
1410.35047326 0.0
1332.69386279 0.0
1256.13107616 0.0
1181.73302426 0.0
1110.21027456 0.0
1042.01540258 0.0
977.456630865 0.0
916.768489842 0.0
860.129624296 0.0
807.646435054 0.0
759.329096224 0.0
715.079775967 0.0
674.700456356 0.0
637.916888936 0.0
604.409802701 0.0
573.844660164 0.0
545.894664779 0.0
520.25553158 0.0
496.653067796 0.0
474.845599933 0.0
454.623222425 0.0
435.80534139 0.0
418.237458266 0.0
401.787751491 0.0
386.343776435 0.0
371.809460737 0.0
358.102477193 0.0
345.152008464 0.0
332.89687339 0.0
321.283962454 0.0
310.266925827 0.0
299.805064335 0.0
289.862385495 0.0
280.406798725 0.0
271.409433302 0.0
262.844068258 0.0
254.686665102 0.0
246.914993312 0.0
239.508336542 0.0
232.447266159 0.0
225.713468646 0.0
219.289614712 0.0
213.159259983 0.0
207.306769426 0.0
201.717259674 0.0
196.376555002 0.

8.29278878857 20.0
8.26419512477 20.0
8.2358043798 20.0
8.20761450929 20.0
8.179623499 20.0
8.15182936423 20.0
8.12423014918 20.0
8.09682392643 20.0
8.06960879629 20.0
8.04258288629 20.0
8.01574435056 20.0
7.98909136932 20.0
7.96262214829 20.0
7.9363349182 20.0
7.9102279342 20.0
7.88429947542 20.0
7.85854784437 20.0
7.83297136651 20.0
7.80756838973 20.0
7.78233728387 20.0
7.75727644023 20.0
7.73238427116 20.0
7.70765920954 20.0
7.68309970839 20.0
7.65870424041 20.0
7.63447129756 20.0
7.61039939064 20.0
7.5864870489 20.0
7.56273281963 20.0
7.53913526778 20.0
7.51569297558 20.0
7.49240454216 20.0
7.46926858321 20.0
7.44628373062 20.0
7.42344863213 20.0
7.40076195101 20.0
7.37822236572 20.0
7.35582856962 20.0
7.33357927064 20.0
7.31147319098 20.0
7.28950906684 20.0
7.2676856481 20.0
7.24600169808 20.0
7.22445599329 20.0
7.2030473231 20.0
7.18177448956 20.0
7.16063630713 20.0
7.13963160243 20.0
7.11875921404 20.0
7.09801799227 20.0
7.07740679893 20.0
7.05692450714 20.0
7.03657000113 20.0
7

3.94367495779 26.666666666666668
3.93745848368 26.666666666666668
3.93125752471 26.666666666666668
3.9250719994 26.666666666666668
3.91890182701 26.666666666666668
3.91274692748 26.666666666666668
3.90660722144 26.666666666666668
3.90048263023 26.666666666666668
3.89437307587 26.666666666666668
3.88827848107 26.666666666666668
3.88219876922 26.666666666666668
3.87613386438 26.666666666666668
3.8700836913 26.666666666666668
3.86404817538 26.666666666666668
3.85802724272 26.666666666666668
3.85202082007 26.666666666666668
3.84602883486 26.666666666666668
3.84005121516 26.666666666666668
3.83408788974 26.666666666666668
3.82813878799 26.666666666666668
3.82220383999 26.666666666666668
3.81628297647 26.666666666666668
3.81037612882 26.666666666666668
3.80448322907 26.666666666666668
3.79860420992 26.666666666666668
3.79273900471 26.666666666666668
3.78688754743 26.666666666666668
3.78104977273 26.666666666666668
3.77522561589 26.666666666666668
3.76941501283 26.666666666666668
3.7636179001

2.74435013377 26.666666666666668
2.74071441042 26.666666666666668
2.73708602909 26.666666666666668
2.73346497688 26.666666666666668
2.72985124097 26.666666666666668
2.72624480859 26.666666666666668
2.72264566704 26.666666666666668
2.71905380365 26.666666666666668
2.71546920583 26.666666666666668
2.71189186103 26.666666666666668
2.70832175675 26.666666666666668
2.70475888051 26.666666666666668
2.70120321991 26.666666666666668
2.69765476254 26.666666666666668
2.69411349607 26.666666666666668
2.69057940818 26.666666666666668
2.68705248657 26.666666666666668
2.68353271899 26.666666666666668
2.68002009319 26.666666666666668
2.67651459698 26.666666666666668
2.67301621815 26.666666666666668
2.66952494453 26.666666666666668
2.66604076395 26.666666666666668
2.66256366428 26.666666666666668
2.65909363337 26.666666666666668
2.6556306591 26.666666666666668
2.65217472936 26.666666666666668
2.64872583202 26.666666666666668
2.64528395499 26.666666666666668
2.64184908615 26.666666666666668
2.638421213

2.03158025179 26.666666666666668
2.02938866479 26.666666666666668
2.02720131703 26.666666666666668
2.02501819767 26.666666666666668
2.02283929591 26.666666666666668
2.02066460103 26.666666666666668
2.01849410239 26.666666666666668
2.01632778941 26.666666666666668
2.0141656516 26.666666666666668
2.01200767852 26.666666666666668
2.00985385983 26.666666666666668
2.00770418524 26.666666666666668
2.00555864458 26.666666666666668
2.00341722772 26.666666666666668
2.00127992463 26.666666666666668
1.99914672537 26.666666666666668
1.99701762005 26.666666666666668
1.99489259889 26.666666666666668
1.99277165219 26.666666666666668
1.99065477032 26.666666666666668
1.98854194376 26.666666666666668
1.98643316303 26.666666666666668
1.98432841878 26.666666666666668
1.98222770172 26.666666666666668
1.98013100263 26.666666666666668
1.9780383124 26.666666666666668
1.97594962199 26.666666666666668
1.97386492243 26.666666666666668
1.97178420485 26.666666666666668
1.96970746043 26.666666666666668
1.9676346804

1.5886801429 26.666666666666668
1.58725237321 26.666666666666668
1.58582636683 26.666666666666668
1.58440211648 26.666666666666668
1.58297961496 26.666666666666668
1.58155885514 26.666666666666668
1.58013982994 26.666666666666668
1.5787225324 26.666666666666668
1.57730695559 26.666666666666668
1.57589309268 26.666666666666668
1.57448093691 26.666666666666668
1.57307048158 26.666666666666668
1.57166172007 26.666666666666668
1.57025464583 26.666666666666668
1.56884925237 26.666666666666668
1.5674455333 26.666666666666668
1.56604348225 26.666666666666668
1.56464309295 26.666666666666668
1.56324435918 26.666666666666668
1.5618472748 26.666666666666668
1.56045183372 26.666666666666668
1.55905802992 26.666666666666668
1.55766585743 26.666666666666668
1.55627531034 26.666666666666668
1.55488638282 26.666666666666668
1.55349906906 26.666666666666668
1.55211336335 26.666666666666668
1.55072925999 26.666666666666668
1.54934675335 26.666666666666668
1.54796583787 26.666666666666668
1.54658650801 

1.27522944815 26.666666666666668
1.27413232016 26.666666666666668
1.27303646336 26.666666666666668
1.27194187786 26.666666666666668
1.2708485637 26.666666666666668
1.26975652088 26.666666666666668
1.26866574938 26.666666666666668
1.26757624914 26.666666666666668
1.26648802004 26.666666666666668
1.26540106194 26.666666666666668
1.26431537465 26.666666666666668
1.26323095796 26.666666666666668
1.2621478116 26.666666666666668
1.26106593528 26.666666666666668
1.25998532866 26.666666666666668
1.25890599137 26.666666666666668
1.25782792299 26.666666666666668
1.25675112307 26.666666666666668
1.25567559115 26.666666666666668
1.25460132669 26.666666666666668
1.25352832913 26.666666666666668
1.25245659789 26.666666666666668
1.25138613234 26.666666666666668
1.25031693181 26.666666666666668
1.24924899561 26.666666666666668
1.24818232301 26.666666666666668
1.24711691323 26.666666666666668
1.24605276549 26.666666666666668
1.24498987895 26.666666666666668
1.24392825274 26.666666666666668
1.2428678859

1.02827258557 33.33333333333333
1.02746024987 33.33333333333333
1.02664881702 33.33333333333333
1.02583828605 33.33333333333333
1.025028656 33.33333333333333
1.02421992591 33.33333333333333
1.02341209482 33.33333333333333
1.02260516178 33.33333333333333
1.02179912584 33.33333333333333
1.02099398605 33.33333333333333
1.02018974148 33.33333333333333
1.01938639119 33.33333333333333
1.01858393424 33.33333333333333
1.01778236971 33.33333333333333
1.01698169667 33.33333333333333
1.01618191419 33.33333333333333
1.01538302136 33.33333333333333
1.01458501725 33.33333333333333
1.01378790096 33.33333333333333
1.01299167157 33.33333333333333
1.01219632818 33.33333333333333
1.01140186988 33.33333333333333
1.01060829577 33.33333333333333
